In [2]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time

import sys
sys.path.insert(0, '../')
import pandas as pd
from datetime import timedelta

In [3]:
from config import key_id, secret_key

In [4]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [5]:
#The url at the end is what makes a paper trading bot
api = tradeapi.REST(key_id, secret_key)

In [6]:
account = api.get_account()
account.daytrading_buying_power

'0'

In [7]:
today = datetime.datetime.today().date() # or datetime.now to use local timezone
daystart=pd.Timestamp(year=today.year, month=today.month, day=today.day, hour=0,tz='America/New_York').isoformat()

In [8]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [9]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'243.1866'

In [10]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [11]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [ ]:
#assetsymb=[]
trade=True
while trade==True:
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    
    assetlist=api.list_positions()
    activity_list=api.get_activities()
    banlist=[]
    oklist=[]
    for activity in  activity_list:
        try:
            if activity.side=='buy':
                if activity.transaction_time.date() == api.get_clock().timestamp.date():
                    banlist.append(activity.symbol)
                    #assetsymb.append(activity.symbol)
        except:
            e=0
    for asset in assetlist:
        
        if asset.symbol not in banlist:
            try:
                
                returned_data = api.get_barset(asset.symbol,barTimeframe,limit=150,start=daystart).df
                    #api.get_barset(symbol,barTimeframe,limit=150).df


                    # Calculated trading indicators
                data=populateindicators(returned_data[asset.symbol])
                
                if data['rsi'][-1] > 71:
                    
                    openPosition = api.get_position(asset.symbol)
                    

                    # Limit order to fully close position
                    returned = api.submit_order(asset.symbol,int(openPosition.qty),"sell","limit","day",data['close'][-1]) 
                    print('Sell',returned)
                    #data['high']

                if float(asset.unrealized_plpc) > .008:
                    if data['plusdi'][-1] < 55 and data['minusdi'][-1] > 45:
                        openPosition = api.get_position(asset.symbol)

                        returned = api.submit_order(asset.symbol,int(openPosition.qty),"sell","limit","day",data['open'][-1]) # Market order to fully close position
                        print('Sell',returned)
            except:
                print(asset.symbol)
                
                
    if currenthour >= 12: 
        trade=False

    #interval number of seconds to wait
    time.sleep(23)


C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [24]:
activity_list[0].transaction_time.date()

datetime.date(2020, 4, 30)